In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import Flatten,Conv2D,Dense,Dropout, MaxPooling2D, Average
import random
import numpy as np

### Loading data

In [2]:
path = "../DATA/Dataset 1-dermnet"
target = (224,224)
batches = 256
gen = ImageDataGenerator(rescale = 1/255.0, validation_split = 0.2)
train = gen.flow_from_directory(path + "/train",
                                target_size = target,
                                batch_size=batches,
                                seed = 106,
                                
                                class_mode="categorical",
                                subset = "training"
                               )
val = gen.flow_from_directory(path + "/train",
                                target_size = target,
                                batch_size=batches,
                                seed = 106,
                                class_mode="categorical",
                                subset = "validation",
                                shuffle = False,
                               )
gen = ImageDataGenerator(rescale = 1/255.0,)
test = gen.flow_from_directory(path + "/test",
                               target_size = target,
                               batch_size=batches,
                               seed = 106,
                               class_mode="categorical",
                               )

Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


## Constants

In [148]:
gene_num = 4
chromosome_num = 6
crossover_rate = 0.25
mutation_rate = 0.2

### Defining The helper functions

In [98]:
def generate(models):
    chrome = []
    for i in range(gene_num):
        chrome.append(random.randint(0,len(models)-1))
    return chrome

In [115]:
def fitness(chromosomes, val):
    top5 = 0
    Y_test = val.labels
    avg = []
    for i in chromosomes:
        avg.append(predictions[i])
    avg = np.array(avg)
    avg = np.average(avg, axis = 0)
   # print(avg.shape)
    for i in range(len(Y_test)):
        if Y_test[i] in np.argsort(avg[i])[-5:]:
            top5 += 1
    
    fit_val= top5/len(Y_test)
    return fit_val

In [105]:

def selection(score, chromosomes):
    # done through the random roulette method
    fitness_sum = sum(score)
    random_val = random.uniform(0, fitness_sum)
    
    # getting cumulative sum till the sum is greater than the random val
    cumulative_sum = 0
    for i in range(0,len(chromosomes)):
        cumulative_sum += score[i]
        if(cumulative_sum>=random_val):
            parent = chromosomes[i]
            # removing the chosen chromosome so that it is not picked again
            del chromosomes[i]
            del score[i]
            return parent

In [106]:
def crossover(parents, cross_rate):
    p1 = parents[0].copy()
    p2 = parents[1].copy()
    if random.random() < cross_rate:
        cross_point = random.randint(0,gene_num-1)
        child1 = p1[ :cross_point] + p2[cross_point:]
        child2 = p2[ :cross_point] + p1[cross_point:]
        return[child1,child2]
    else:
        return [p1,p2]

In [107]:
def mutation(chromosome, mutation_rate,models):
    for i in range(gene_num):
        if random.random() < mutation_rate:
            mutation_point = random.randint(0,gene_num-1)
            mutation_val = random.randint(0,len(models)-1)
            chromosome[mutation_point] = mutation_val
    return chromosome

### Defining the genetic algorithm

In [146]:
def genetic_algo(iterations,models,validation_data):
    # defining the empty chromosomes list
    chromosomes = []
    best_gen = 0
    best_gen_val = 0
    # creating the initial chromosomes
    for i in range(0,chromosome_num):
        chromosomes.append(generate(models))
    print("The initial chromosomes made: \n")
    print(chromosomes)
    # running the program to make new generations for the given iterations
    for k in range(iterations):
        current_gen_scores = []
        parents = []
        next_gen = []
        for i in chromosomes:
            current_gen_scores.append(fitness(i,validation_data))
        
        if(max(current_gen_scores)> best_gen_val):
            best_gen_val = max(current_gen_scores)
            best_gen = chromosomes[current_gen_scores.index(best_gen_val)]
     
        # stopping criteria
        if 1.0 in current_gen_scores:
            break
        
        # getting the two parents allowed to mate for the next generation
        parents.append(selection(current_gen_scores, chromosomes))
        parents.append(selection(current_gen_scores, chromosomes))
        
        # making the new generation
        for i in range(int(chromosome_num/2)):
            for child in crossover(parents, crossover_rate):
                next_gen += [mutation(child,mutation_rate,models)]
        chromosomes = next_gen.copy()
        
    # printing the final chromosomes along with their values and also the best chromosome
    best_chromosome = 0
    best_chromosome_value = 0
    print("The final generation chromosomes are:\n", chromosomes, "\n")
    for i in range(len(chromosomes)):
        fit_val = fitness(chromosomes[i],validation_data)
        print(f"Chromosome {i+1}: ", chromosomes[i], "top 5 accuracy : ",fit_val )
        if(fit_val > best_chromosome_value):
            best_chromosome = i
            best_chromosome_value = fit_val
    print("\nThe best chromosome is: ", chromosomes[best_chromosome], " with value: ", best_chromosome_value)
    
    # if the solution was found we also print the generation which gave the solution
    if(k != iterations-1):
        print("Generation that produced the answer is: ", k+1)
    print("=-------------------")
    print(best_gen)
    print(best_gen_val)

### Loading models

In [10]:
model_path= "/Users/praksj/Documents/VIT/Year-3/sem 6/TARP/DATA/dermnet_saves/models/"
resnet50v2 = load_model(model_path +"resnet50v2.h5" ) #0
densenet_unfrozen = load_model(model_path +"densenet_unfrozen.h5" ) #1
base_cnn = load_model(model_path + "base_cnn.h5") #2
resnet50 = load_model(model_path +"resnet.h5" ) #3
resnet101 = load_model(model_path +"resnet101.h5" ) #4 
efficientnet_unfrozen = load_model(model_path + "efficient_netV2_unfrozen.h5") #5
models = [resnet50v2,densenet_unfrozen,base_cnn,resnet50,resnet101,efficientnet_unfrozen]

In [143]:

predictions = []
for model in models:
    pred = model.predict(val)
    predictions.append(pred)

2023-02-15 00:33:44.549222: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 37s 3s/step


2023-02-15 00:34:22.999033: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 51s 3s/step


2023-02-15 00:35:14.235886: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

13/13 [==============================] - 43s 3s/step


2023-02-15 00:36:09.883080: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 72s 5s/step


2023-02-15 00:37:24.784869: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 64s 4s/step


## Running the Genetic Algorithm

In [150]:
genetic_algo(100,models,val)

The initial chromosomes made: 

[[0, 4, 0, 3], [0, 3, 3, 5], [0, 0, 4, 0], [0, 1, 0, 4], [2, 2, 5, 4], [1, 0, 3, 1]]
The final generation chromosomes are:
 [[2, 3, 0, 1], [2, 3, 0, 1], [2, 3, 5, 1], [2, 4, 0, 1], [2, 3, 1, 3], [2, 4, 0, 1]] 

Chromosome 1:  [2, 3, 0, 1] top 5 accuracy :  0.5934278350515464
Chromosome 2:  [2, 3, 0, 1] top 5 accuracy :  0.5934278350515464
Chromosome 3:  [2, 3, 5, 1] top 5 accuracy :  0.5657216494845361
Chromosome 4:  [2, 4, 0, 1] top 5 accuracy :  0.5676546391752577
Chromosome 5:  [2, 3, 1, 3] top 5 accuracy :  0.5708762886597938
Chromosome 6:  [2, 4, 0, 1] top 5 accuracy :  0.5676546391752577

The best chromosome is:  [2, 3, 0, 1]  with value:  0.5934278350515464
=-------------------
[2, 3, 0, 2]
0.5940721649484536


## Evaluating the both the final generation fittest model and the fittest model through generations

#### Final gen

In [11]:
model_indexes = [2, 3, 0, 1]
models_chosen= []
for i in model_indexes:
    models_chosen.append(models[i])
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models_chosen]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output)

In [12]:
ensemble_model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy", "top_k_categorical_accuracy"])


In [13]:
ensemble_model.evaluate(test)

2023-02-15 01:21:30.874001: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 220s 14s/step - loss: 2.0189 - accuracy: 0.4545 - top_k_categorical_accuracy: 0.7339


[2.0189249515533447, 0.4545227587223053, 0.7338830828666687]

#### Best gen

In [14]:
model_indexes = [2, 3, 0, 2]
models_chosen= []
for i in model_indexes:
    models_chosen.append(models[i])
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models_chosen]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output)
ensemble_model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy", "top_k_categorical_accuracy"])


In [15]:
ensemble_model.evaluate(test)

2023-02-15 01:28:47.542509: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 447s 28s/step - loss: 2.1468 - accuracy: 0.4033 - top_k_categorical_accuracy: 0.7049


[2.1467671394348145, 0.40329834818840027, 0.7048975825309753]

# Sanity check

In [5]:
model_path= "/Users/praksj/Documents/VIT/Year-3/sem 6/TARP/DATA/dermnet_saves/models/"
resnet50v2 = load_model(model_path +"resnet50v2.h5" ) 
densenet_unfrozen = load_model(model_path +"densenet_unfrozen.h5" ) 
base_cnn = load_model(model_path + "base_cnn.h5")

Metal device set to: Apple M1


2023-02-15 01:18:16.060699: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-15 01:18:16.060824: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
models = [resnet50v2, densenet_unfrozen,base_cnn]
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output)

In [7]:
ensemble_model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy", "top_k_categorical_accuracy"])


In [8]:
ensemble_model.evaluate(test)

2023-02-15 01:18:22.940145: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-15 01:18:24.449654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 100s 6s/step - loss: 2.1539 - accuracy: 0.4538 - top_k_categorical_accuracy: 0.7354


[2.15393328666687, 0.4537731111049652, 0.7353823184967041]